# Dirty Road PAtch (DRP) attack on MetaDrive with CNN-based lane detection

References:
- https://github.com/ASGuard-UCI/DRP-attack

In [20]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

import cv2
import numpy as np

In [29]:
from pytorch_auto_drive.utils.lr_schedulers.builder import LR_SCHEDULERS
from pytorch_auto_drive.utils.common import load_checkpoint
from pytorch_auto_drive.utils.args import read_config
from pytorch_auto_drive.utils.models import MODELS
from pytorch_auto_drive.utils.losses import LOSSES
from pytorch_auto_drive.utils.runners.base import BaseTrainer
import torch


CONFIG=os.path.dirname(os.getcwd()) + '/pytorch_auto_drive/configs/lane_detection/resa/resnet50_culane.py'
cfg = read_config(CONFIG)
model = MODELS.from_dict(cfg['model'])

optimizer = BaseTrainer.get_optimizer(cfg['optimizer'], model)
# lr_scheduler = LR_SCHEDULERS.from_dict(cfg['lr_scheduler'], model, len(cfg['train_loader'])
loss = LOSSES.from_dict(cfg['loss'])
num_classes = cfg['train']['num_classes']
input_size = cfg['train']['input_size']

load_checkpoint(net=model, optimizer=optimizer, lr_scheduler=None, filename=os.path.dirname(os.getcwd()) + '/../../resnet50_resa_culane_20211016.pt', strict=False)




Loaded torchvision ImageNet pre-trained weights V1.
Build from dict error in function or class: LaneLoss
In Python: <class 'pytorch_auto_drive.utils.losses.lane_seg_loss.LaneLoss'>


AssertionError: Torch not compiled with CUDA enabled

In [30]:
# create a classifier
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
import pytorch_auto_drive.functional as F
from PIL import Image

classifier = PyTorchClassifier(
    model=model,
    loss=loss,
    optimizer=optimizer,
    input_shape=(1, input_size[0], input_size[1]),
    nb_classes=num_classes,
)

image = Image.open('../../lanefitting/example_input.jpg')
orig_sizes = (image.height, image.width)
original_img = F.to_tensor(image).clone().unsqueeze(0)
image = F.resize(image, size=input_size)

model_in = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))

model_in = model_in.view(image.size[1], image.size[0], len(image.getbands()))
model_in = (
    model_in.permute((2, 0, 1)).contiguous().float().div(255).unsqueeze(0).numpy()
)

attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=model_in)

NameError: name 'loss' is not defined

In [22]:
from art.attacks.evasion import RobustDPatch

detector = model # TODO: implement a detector

attack = RobustDPatch(
    detector,
    patch_shape=(3, 200, 200),
    patch_location=(50, 50),
    crop_range=[0,0],
    brightness_range=[1.0, 1.0],
    rotation_weights=[1, 0, 0, 0],
    sample_size=1,
    learning_rate=1.99,
    max_iter=1,
    batch_size=1,
    verbose=False,
    targeted=True
)

EstimatorError: RobustDPatch requires an estimator derived from <class 'art.estimators.estimator.BaseEstimator'> and <class 'art.estimators.estimator.LossGradientsMixin'> and <class 'art.estimators.object_detection.object_detector.ObjectDetectorMixin'>, the provided classifier is an instance of <class 'pytorch_auto_drive.utils.models.lane_detection.resa.RESA_Net'> and is derived from (<class 'torch.nn.modules.module.Module'>,).